In [2]:
from setup_general import *

# Delete unneeded features


In [9]:
data = prep.copy()
data.drop(columns=['full_nr', 'name','commentary','country_and_unit','text','parameter','unit','value','legend','initial_info','additional_text'], inplace=True)
data.to_csv('data/prep.csv')

In [13]:
data = prep.copy()
data.drop(columns=['film frame IN mm²', 'photo format IN mm²', 'negative format IN mm²', 'text_features'], inplace=True)
data.to_csv('data/prep.csv')

# Nan ->0

In [7]:
data = prep.copy()

#for col in data.columns:
 #   if col.hasnan:
  #      data[col] = data[col].replace(np.nan, 0)

data = data.replace(np.nan, 0)
print (data.isnull().sum().sum())
print (data)

0
              ks  musealia_seria_nr  musealia_queue_nr  collection_queue_nr  \
id                                                                            
232170     118.0            12150.0              115.0                 44.0   
2251378    102.0            12584.0               19.0                  1.0   
2070466      0.0                0.0                0.0                 52.0   
4085096  41691.0                0.0                0.0                  0.0   
2697904    238.0             9424.0                0.0                  1.0   
...          ...                ...                ...                  ...   
3035795     10.0             5429.0                0.0                  3.0   
2765287      0.0            12656.0               13.0                 13.0   
2416520    429.0             8293.0                0.0                435.0   
2327407    580.0            29884.0                0.0               5037.0   
2781747  13349.0                0.0               

In [8]:
data.to_csv('data/prep.csv')

# remove un-thresholded features


In [7]:
data = prep.copy()

tech = helpers.col_collection(data, 'technique_')
mat = helpers.col_collection(data, 'material_')
loc = helpers.col_collection(data, 'location_')
cols = tech + mat + loc

for col in cols:
    if data[col].sum() < 2:
        data.drop(columns=[col], inplace=True)

82 columns found that start with technique_
84 columns found that start with material_
74 columns found that start with location_


# resplit test/train


In [10]:
data = prep.copy()

train = data.loc[data['source']=='train'].drop('source',axis=1)
test = data.loc[data['source']=='test'].drop('source',axis=1)

train.to_csv('data/train_prepared.csv')
test.to_csv('data/test_prepared.csv')